### 1. Data Import and Base Variable 확인

 * id : 식별자 for each trip
 * vendor_id : 공급자 코드(a code indicating the provider associated with the trip record)
 * pickup_datetime : 탑승 날짜 및 시각(date and time when the meter was engaged)
 * dropoff_datetime : 하차 날짜 및 시각(date and time when the meter was disengaged)
 * passengers in the vehicle : 승객수(driver entered value)
 * pickup_longitude : 탑승 경도(the longitude where the meter was engaged)
 * pickup_latitude : 탑승 위도(the latitude where the meter was engaged)
 * dropoff_longitude : 하차 경도(the longitude where the meter was disengaged)
 * dropoff_latitude : 하차 위도(the latitude where the meter was disengaged)
 * store_and_fwd_flag : 차량메모리 보관여부(this flage indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server. Y = store and forward, N = not a store and forward trip)
 * trip_duration : duration of the trip in seconds

In [1]:
# notebook파일 생성한 폴더에 csv파일을 옮겨두고 data import
taxi = 'train.csv'

In [2]:
# dataframe 생성
taxi_df = pd.read_csv(taxi)

In [6]:
# dataframe의 최초 값 확인 
taxi_df.head(100)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435
5,id0801584,2,2016-01-30 22:01:40,2016-01-30 22:09:03,6,-73.982857,40.742195,-73.992081,40.749184,N,443
6,id1813257,1,2016-06-17 22:34:59,2016-06-17 22:40:40,4,-73.969017,40.757839,-73.957405,40.765896,N,341
7,id1324603,2,2016-05-21 07:54:58,2016-05-21 08:20:49,1,-73.969276,40.797779,-73.922470,40.760559,N,1551
8,id1301050,1,2016-05-27 23:12:23,2016-05-27 23:16:38,1,-73.999481,40.738400,-73.985786,40.732815,N,255
9,id0012891,2,2016-03-10 21:45:01,2016-03-10 22:05:26,1,-73.981049,40.744339,-73.973000,40.789989,N,1225


In [5]:
# dataframe의 마지막 값 확인 
taxi_df.tail(50)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
1458594,id3274818,2,2016-03-11 21:04:31,2016-03-11 21:08:41,2,-73.978233,40.763203,-73.982498,40.766701,N,250
1458595,id2313812,2,2016-06-21 12:48:12,2016-06-21 12:55:34,1,-73.974312,40.762798,-73.978531,40.756931,N,442
1458596,id1169625,1,2016-04-19 14:42:08,2016-04-19 14:57:30,1,-73.984756,40.728607,-74.007294,40.704018,N,922
1458597,id2066774,1,2016-04-26 16:11:51,2016-04-26 16:21:26,1,-73.959831,40.779663,-73.978432,40.787720,N,575
1458598,id0018431,1,2016-04-09 00:44:34,2016-04-09 00:53:01,1,-73.982330,40.771317,-73.997917,40.756401,N,507
1458599,id2438336,2,2016-01-21 14:10:36,2016-01-21 14:24:55,1,-73.996323,40.758526,-73.978844,40.744671,N,859
1458600,id0995846,2,2016-05-09 17:26:56,2016-05-09 18:30:37,2,-73.789543,40.647099,-73.960320,40.798180,N,3821
1458601,id1041783,1,2016-06-23 16:14:26,2016-06-23 16:25:09,1,-73.970894,40.764320,-73.969398,40.794540,N,643
1458602,id2089150,1,2016-05-30 14:42:12,2016-05-30 14:42:38,1,-74.015457,40.708027,-74.015396,40.708138,N,26
1458603,id1188619,2,2016-02-29 10:53:26,2016-02-29 10:58:05,1,-74.014740,40.717838,-74.008522,40.724899,N,279


In [4]:
# data가 너무 크기 때문에 일단 dataframe의 raw를 5만개로 줄여서 시작 (근데 뒤에 해보면 50000개 넘는거 같음)
taxi_df[:20000]

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435
5,id0801584,2,2016-01-30 22:01:40,2016-01-30 22:09:03,6,-73.982857,40.742195,-73.992081,40.749184,N,443
6,id1813257,1,2016-06-17 22:34:59,2016-06-17 22:40:40,4,-73.969017,40.757839,-73.957405,40.765896,N,341
7,id1324603,2,2016-05-21 07:54:58,2016-05-21 08:20:49,1,-73.969276,40.797779,-73.922470,40.760559,N,1551
8,id1301050,1,2016-05-27 23:12:23,2016-05-27 23:16:38,1,-73.999481,40.738400,-73.985786,40.732815,N,255
9,id0012891,2,2016-03-10 21:45:01,2016-03-10 22:05:26,1,-73.981049,40.744339,-73.973000,40.789989,N,1225


### 2. Data Pre-Processing
#### 1) rawdata base variable type 확인

In [7]:
taxi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


#### 2) 시간 변수 수정
* pickup_datetime, dropoff_datetime은 type이 datetime이어야 시간변수로 활용할 수 있으나, object(=string)임
* 시간변수로 활용하기 위해 type을 datetime으로 바꿔줘야 함
* datetime함수를 활용해 type을 바꿔줄 수 있음

In [9]:
# 시간변수 type 확인
taxi_df['pickup_datetime'].dtype

dtype('O')

In [10]:
# 시간변수 type을 datetime으로 수정
taxi_df['pickup_datetime'] = pd.to_datetime(taxi_df['pickup_datetime'])
taxi_df['dropoff_datetime'] = pd.to_datetime(taxi_df['dropoff_datetime'])


# pickup - dropoff의 계산을 통해 trip_duration 검증 및 pickup, dropoff, trip_duration 변수 관계 확인
taxi_df['trip_duration_c'] = taxi_df['dropoff_datetime'] - taxi_df['pickup_datetime']


# trip_duration_c는 day 단위로 산출됨. 그러나 trip_duration의 단위는 second임 따라서 day를 second단위로 변환
taxi_df['trip_duration_c'] = taxi_df['trip_duration_c'].dt.total_seconds()

* pickup_datetime의 년, 월, 일을 각각 새로운 변수로 설정함
    * trip_duration과 pickup한 년, 월, 일 별로 상관관계가 있을 것 같아서 새로운 변수를 생성하고자 함
        * ex) 아주 춥거나 더운 달(12, 1, 2, 7, 8월에 택시를 탈 경우 trip_duration이 다른 월보다 길 수 있다고 생각함)
        * 뉴욕의 날씨를 확인해서 춥거나 더운 달과 그렇지 않은 달로 구분해서 더미변수를 만들 경우 설명변수로 사용 가능 할 것으로 판단
    * trip_duration이 pickup_datetime과 dropoff_datetime의 차로 형성된 변수임
        * 따라서  trip_duration를 이용해 새로운 변수를 만들고 그 변수를 설명변수로 사용할 경우 다중공선성이 발생할 가능성이 있음
        * 하지만 pickup_datetime이 시간변수이기 때문에 이에 대해 확신할 수 없어서 일단 해보기로 함
        * pickup_datetime에서 dt.components로 년, 월, 일을 뽑아 낼 수 있는 것으로 나오지만 error 해결 못함
            * taxi_df['pickup_dateime'].dt.components -> error 발생
        * 그래서 부득이 slicing을 사용함 

In [11]:
# taxi_df['pickup_datetime'].dt.components dataframe에서 컴포넌트 하는 방법은 아직 모르겠음 그래서 부득이 slicing을 함


# pickup_datetime을 각 항목별로 slicing (slicing은 str밖에 안되므로 타입을 str로바꿔서 slicing)
taxi_df['pickup_datetime'] = taxi_df['pickup_datetime'].astype(str)
taxi_df['pickup_year'] = taxi_df['pickup_datetime'].str[:4]

# pickup_year의 type을 string에서 int로 다시 바꿔줌
taxi_df['pickup_year'] = taxi_df['pickup_year'].astype(int)

# 위의 방법을 month, day, hour, mm에 반복  (보다 효율적으로 하는 방법(loop등)이 있을 것 같음)
taxi_df['pickup_month'] = taxi_df['pickup_datetime'].str[5:7]
taxi_df['pickup_month'] = taxi_df['pickup_month'].astype(int)
taxi_df['pickup_day'] = taxi_df['pickup_datetime'].str[8:10]
taxi_df['pickup_day'] = taxi_df['pickup_day'].astype(int)
taxi_df['pickup_hour'] = taxi_df['pickup_datetime'].str[10:13]
taxi_df['pickup_hour'] = taxi_df['pickup_hour'].astype(int)
taxi_df['pickup_mm'] = taxi_df['pickup_datetime'].str[14:16]
taxi_df['pickup_mm'] = taxi_df['pickup_mm'].astype(int)

# 생성된 변수 타입 확인
taxi_df.info()

# 생성된 변수 dataframe 적용 확인
taxi_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 17 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null datetime64[ns]
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
trip_duration_c       1458644 non-null float64
pickup_year           1458644 non-null int64
pickup_month          1458644 non-null int64
pickup_day            1458644 non-null int64
pickup_hour           1458644 non-null int64
pickup_mm             1458644 non-null int64
dtypes: datetime64[ns](1), float64(5), int64(8), object(3)
memory usage: 189.2+ MB


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,trip_duration_c,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_mm
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,455.0,2016,3,14,17,24
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,663.0,2016,6,12,0,43
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,2124.0,2016,1,19,11,35
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,429.0,2016,4,6,19,32
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,435.0,2016,3,26,13,30


#### 3) 월별 더미변수 생성
* pickup_month가 6까지 밖에 없음
* 1, 2, 3월은 0, 4, 5, 6월은 1로 dummy변수 생성

In [12]:
# pickup_month data 확인을 위해 grouping

month_group = taxi_df.groupby('pickup_month')

In [13]:
len(month_group.groups)

6

In [16]:
month_group.count()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,trip_duration_c,pickup_year,pickup_day,pickup_hour,pickup_mm
pickup_month,,,,,,,,,,,,,,,,
1,229707,229707,229707,229707,229707,229707,229707,229707,229707,229707,229707,229707,229707,229707,229707,229707
2,238300,238300,238300,238300,238300,238300,238300,238300,238300,238300,238300,238300,238300,238300,238300,238300
3,256189,256189,256189,256189,256189,256189,256189,256189,256189,256189,256189,256189,256189,256189,256189,256189
4,251645,251645,251645,251645,251645,251645,251645,251645,251645,251645,251645,251645,251645,251645,251645,251645
5,248487,248487,248487,248487,248487,248487,248487,248487,248487,248487,248487,248487,248487,248487,248487,248487
6,234316,234316,234316,234316,234316,234316,234316,234316,234316,234316,234316,234316,234316,234316,234316,234316


In [17]:
month_group.size().sort_values(ascending = False)

pickup_month
3    256189
4    251645
5    248487
2    238300
6    234316
1    229707
dtype: int64

In [24]:
if (taxi_df['pickup_month'] = 1) or (taxi_df['pickup_month'] = 2) or (taxi_df['pickup_month'] = 3):
    taxi_df['dummy_month'] = 0

SyntaxError: invalid syntax (<ipython-input-24-fad2b3062256>, line 1)

 #### 2) 공간변수(or 지도정보, 위치정보) 사용 
  * 지도정보(위치정보)를 활용하기 위해서는 folium(포리움)함수를 사용해야 함

In [9]:
#correlation 확인
taxi_df['trip_duration'].corr(taxi_df['pickup_month'])

0.0066071185672692068

In [ ]:
taxi_df.corr()

In [11]:
taxi_df.values

array([['2016-03-14 17:24:55', 'id2875421', 2, ..., 455, '03', '14'],
       ['2016-06-12 00:43:35', 'id2377394', 1, ..., 663, '06', '12'],
       ['2016-01-19 11:35:24', 'id3858529', 2, ..., 2124, '01', '19'],
       ..., 
       ['2016-04-22 06:57:41', 'id2304944', 2, ..., 764, '04', '22'],
       ['2016-01-05 15:56:26', 'id2714485', 1, ..., 373, '01', '05'],
       ['2016-04-05 14:44:25', 'id1209952', 1, ..., 198, '04', '05']], dtype=object)

In [32]:
taxi_df['pickup_month'].value_counts()

03    256189
04    251645
05    248487
02    238300
06    234316
01    229707
Name: pickup_month, dtype: int64

In [6]:
pickup_date_group = taxi_df.groupby(lambda pickup_datetime : pickup_datetime.split(' ')[0])
pickup_date_group.head()

,id,vendor_id,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
pickup_datetime,,,,,,,,,,
2016-03-14 17:24:55,id2875421,2,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
2016-06-12 00:43:35,id2377394,1,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2016-01-19 11:35:24,id3858529,2,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
2016-04-06 19:32:31,id3504673,2,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
2016-03-26 13:30:55,id2181028,2,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435
2016-01-30 22:01:40,id0801584,2,2016-01-30 22:09:03,6,-73.982857,40.742195,-73.992081,40.749184,N,443
2016-06-17 22:34:59,id1813257,1,2016-06-17 22:40:40,4,-73.969017,40.757839,-73.957405,40.765896,N,341
2016-05-21 07:54:58,id1324603,2,2016-05-21 08:20:49,1,-73.969276,40.797779,-73.922470,40.760559,N,1551
2016-05-27 23:12:23,id1301050,1,2016-05-27 23:16:38,1,-73.999481,40.738400,-73.985786,40.732815,N,255


In [8]:
taxi_df.reset_index(inplace=True)

In [9]:
taxi_df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id3495688,1,4/30/16 23:59,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454
1,id3051282,1,4/30/16 23:59,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409
2,id0082851,1,4/30/16 23:59,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081
3,id0515725,2,4/30/16 23:58,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800
4,id1872374,2,4/30/16 23:58,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151


In [10]:
taxi_df.shape

(701778, 11)

In [11]:
taxi_df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id3495688,1,4/30/16 23:59,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454
1,id3051282,1,4/30/16 23:59,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409
2,id0082851,1,4/30/16 23:59,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081
3,id0515725,2,4/30/16 23:58,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800
4,id1872374,2,4/30/16 23:58,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151


In [12]:
taxi_df.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,701778.000000,701778.000000,701778.000000,701778.000000,701778.000000,701778.000000,7.017780e+05
mean,1.536037,1.666577,-73.973665,40.750969,-73.973583,40.751836,9.387318e+02
std,0.498700,1.319466,0.043910,0.033139,0.044214,0.037572,6.772504e+03
min,1.000000,0.000000,-78.547401,34.359695,-79.817978,32.181141,1.000000e+00
25%,1.000000,1.000000,-73.991829,40.737400,-73.991325,40.735959,3.900000e+02
50%,2.000000,1.000000,-73.981789,40.754150,-73.979828,40.754478,6.480000e+02
75%,2.000000,2.000000,-73.967567,40.768246,-73.963249,40.769691,1.047000e+03
max,2.000000,8.000000,-61.335529,43.486885,-61.335529,43.674000,3.526282e+06


In [13]:
taxi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701778 entries, 0 to 701777
Data columns (total 11 columns):
id                    701778 non-null object
vendor_id             701778 non-null int64
pickup_datetime       701778 non-null object
dropoff_datetime      701778 non-null object
passenger_count       701778 non-null int64
pickup_longitude      701778 non-null float64
pickup_latitude       701778 non-null float64
dropoff_longitude     701778 non-null float64
dropoff_latitude      701778 non-null float64
store_and_fwd_flag    701778 non-null object
trip_duration         701778 non-null int64
dtypes: float64(4), int64(3), object(4)
memory usage: 58.9+ MB


In [14]:
dfy = taxi_df.ix[:,10:11]
dfX = taxi_df.ix[:,:10]
print(len(dfy.columns), len(dfX.columns))

1 10


/home/dockeruser/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [15]:
dfy.tail()

,trip_duration
701773,280
701774,408
701775,736
701776,849
701777,1294


In [16]:
dfX.tail()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
701773,id3888279,1,1/1/16 0:01,1/1/16 0:05,1,-73.982292,40.751331,-73.991341,40.750340,N
701774,id1210365,2,1/1/16 0:01,1/1/16 0:07,5,-73.965279,40.801041,-73.947479,40.815170,N
701775,id0924227,1,1/1/16 0:01,1/1/16 0:13,1,-73.970108,40.759800,-73.989357,40.742989,N
701776,id0190469,2,1/1/16 0:00,1/1/16 0:14,5,-73.981743,40.719158,-73.938828,40.829182,N
701777,id1665586,1,1/1/16 0:00,1/1/16 0:22,1,-73.985085,40.747166,-73.958038,40.717491,N


In [17]:
cols_real = ['pickup_datetime', 'dropoff_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']

In [18]:
cols_cat = ['id', 'vendor_id', 'store_and_fwd_flag']

In [19]:
dfX_real = dfX[cols_real]
print(len(dfX_real.columns))
dfX_real.tail()

7


,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
701773,1/1/16 0:01,1/1/16 0:05,1,-73.982292,40.751331,-73.991341,40.750340
701774,1/1/16 0:01,1/1/16 0:07,5,-73.965279,40.801041,-73.947479,40.815170
701775,1/1/16 0:01,1/1/16 0:13,1,-73.970108,40.759800,-73.989357,40.742989
701776,1/1/16 0:00,1/1/16 0:14,5,-73.981743,40.719158,-73.938828,40.829182
701777,1/1/16 0:00,1/1/16 0:22,1,-73.985085,40.747166,-73.958038,40.717491


In [20]:
dfX_cat = dfX[cols_cat]
print(len(dfX_cat.columns))
dfX_cat.tail()

3


,id,vendor_id,store_and_fwd_flag
701773,id3888279,1,N
701774,id1210365,2,N
701775,id0924227,1,N
701776,id0190469,2,N
701777,id1665586,1,N


In [21]:
print(plt.style.available)

['seaborn-white', 'seaborn', 'grayscale', 'ggplot', 'seaborn-darkgrid', 'seaborn-paper', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-deep', 'seaborn-whitegrid', 'seaborn-poster', 'dark_background', 'seaborn-dark', 'bmh', 'classic', 'seaborn-notebook', 'seaborn-talk', '_classic_test', 'fivethirtyeight', 'seaborn-pastel', 'seaborn-muted', 'seaborn-ticks', 'seaborn-bright']


In [22]:
for i in range(0, len(cols_real), 4):
    print("dfX{}".format(cols_real[i: i+4]))

dfX['pickup_datetime', 'dropoff_datetime', 'passenger_count', 'pickup_longitude']
dfX['pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']


In [33]:
sns.barplot(x="pickup_datetime", y="trip_duration", data=taxi_df)

KeyboardInterrupt: 